In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import random
import numpy as np
import torch

# Fix random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
from torch.utils.data import DataLoader

train_dir = "/content/drive/MyDrive/Frames_sequential_split/train"
test_dir  = "/content/drive/MyDrive/Frames_sequential_split/test"

train_dataset = SequenceDataset(train_dir, transform=transform, seq_len=30)
test_dataset  = SequenceDataset(test_dir, transform=transform, seq_len=30)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=4, shuffle=False)

print(f"Train clips: {len(train_dataset)}, Test clips: {len(test_dataset)}")


NameError: name 'SequenceDataset' is not defined

In [ ]:
# ========================
# 1️⃣ Setup
# ========================
!pip install timm --quiet

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

# ========================
# 2️⃣ Data augmentation
# ========================
class VideoTransform:
    def __init__(self, train=True):
        if train:
            self.transform = transforms.Compose([
                transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5),
                                     (0.25, 0.25, 0.25))
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5),
                                     (0.25, 0.25, 0.25))
            ])

    def __call__(self, frames):
        return torch.stack([self.transform(f) for f in frames], dim=0)  # (seq_len, 3, 224, 224)


# ========================
# 3️⃣ Dataset class
# ========================
class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None, seq_len=16):
        self.root_dir = root_dir
        self.transform = transform
        self.seq_len = seq_len
        self.samples = []

        for label in ['0', '1']:  # two classes
            class_path = os.path.join(root_dir, label)
            if not os.path.exists(class_path):
                continue
            for clip_folder in os.listdir(class_path):
                clip_path = os.path.join(class_path, clip_folder)
                if os.path.isdir(clip_path):
                    self.samples.append((clip_path, int(label)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        folder_path, label = self.samples[idx]
        frame_files = sorted(os.listdir(folder_path))[:self.seq_len]
        frames = [Image.open(os.path.join(folder_path, f)).convert("RGB") for f in frame_files]
        frames = self.transform(frames)
        return frames, torch.tensor(label, dtype=torch.float32)


# ========================
# 4️⃣ Load data
# ========================
train_dir = "/content/drive/MyDrive/Frames_sequential_split/train"
test_dir  = "/content/drive/MyDrive/Frames_sequential_split/test"

train_dataset = VideoDataset(train_dir, transform=VideoTransform(train=True))
test_dataset  = VideoDataset(test_dir, transform=VideoTransform(train=False))

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=2)

print(f"✅ Train samples: {len(train_dataset)}")
print(f"✅ Test samples: {len(test_dataset)}")


# ========================
# 5️⃣ CNN + Transformer Model
# ========================
class CNN_Transformer(nn.Module):
    def __init__(self, cnn_embed_dim=512, num_heads=8, num_layers=2, num_classes=1):
        super(CNN_Transformer, self).__init__()
        # CNN backbone
        self.cnn = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT).features
        self.cnn_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.cnn_fc = nn.Linear(1280, cnn_embed_dim)

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=cnn_embed_dim,
            nhead=num_heads,
            dim_feedforward=1024,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Classification head
        self.fc = nn.Sequential(
            nn.Linear(cnn_embed_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x: (batch, seq_len, 3, H, W)
        b, s, c, h, w = x.size()
        x = x.view(b * s, c, h, w)
        with torch.no_grad():
            f = self.cnn(x)
        f = self.cnn_pool(f).view(b * s, -1)
        f = self.cnn_fc(f)
        f = f.view(b, s, -1)
        t_out = self.transformer(f)
        out = self.fc(t_out[:, -1, :])
        return out


# ========================
# 6️⃣ Training setup
# ========================
model = CNN_Transformer().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10
best_acc = 0.0


# ========================
# 7️⃣ Training loop
# ========================
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for frames, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        frames, labels = frames.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(frames).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_correct, val_total, val_loss = 0, 0, 0
    with torch.no_grad():
        for frames, labels in test_loader:
            frames, labels = frames.to(device), labels.to(device)
            outputs = model(frames).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = (outputs > 0.5).float()
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_loss /= len(test_loader)

    print(f"\nEpoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_cnn_transformer.pth")
        print("✅ Saved best model")

print(f"\n🎯 Training complete! Best Validation Accuracy: {best_acc:.2f}%")


✅ Device: cuda
✅ Train samples: 61
✅ Test samples: 16
Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 143MB/s]
Epoch 1/10:  97%|█████████▋| 30/31 [04:04<00:08,  8.16s/it]


ValueError: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.

In [ ]:
print(outputs.shape, labels.shape)


torch.Size([]) torch.Size([1])


In [ ]:
# ========================
# 1️⃣ Setup
# ========================
!pip install timm --quiet

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✅ Device:", device)

# ========================
# 2️⃣ Data augmentation
# ========================
class VideoTransform:
    def __init__(self, train=True):
        if train:
            self.transform = transforms.Compose([
                transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5),
                                     (0.25, 0.25, 0.25))
            ])
        else:
            self.transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5),
                                     (0.25, 0.25, 0.25))
            ])

    def __call__(self, frames):
        return torch.stack([self.transform(f) for f in frames], dim=0)  # (seq_len, 3, 224, 224)


# ========================
# 3️⃣ Dataset class
# ========================
class VideoDataset(Dataset):
    def __init__(self, root_dir, transform=None, seq_len=16):
        self.root_dir = root_dir
        self.transform = transform
        self.seq_len = seq_len
        self.samples = []

        for label in ['0', '1']:  # two classes
            class_path = os.path.join(root_dir, label)
            if not os.path.exists(class_path):
                continue
            for clip_folder in os.listdir(class_path):
                clip_path = os.path.join(class_path, clip_folder)
                if os.path.isdir(clip_path):
                    self.samples.append((clip_path, int(label)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        folder_path, label = self.samples[idx]
        frame_files = sorted(os.listdir(folder_path))[:self.seq_len]
        frames = [Image.open(os.path.join(folder_path, f)).convert("RGB") for f in frame_files]
        frames = self.transform(frames)
        return frames, torch.tensor(label, dtype=torch.float32)


# ========================
# 4️⃣ Load data
# ========================
train_dir = "/content/drive/MyDrive/Frames_sequential_split/train"
test_dir  = "/content/drive/MyDrive/Frames_sequential_split/test"

train_dataset = VideoDataset(train_dir, transform=VideoTransform(train=True))
test_dataset  = VideoDataset(test_dir, transform=VideoTransform(train=False))

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=2)

print(f"✅ Train samples: {len(train_dataset)}")
print(f"✅ Test samples: {len(test_dataset)}")


# ========================
# 5️⃣ CNN + Transformer Model
# ========================
class CNN_Transformer(nn.Module):
    def __init__(self, cnn_embed_dim=512, num_heads=8, num_layers=2, num_classes=1):
        super(CNN_Transformer, self).__init__()
        # CNN backbone
        self.cnn = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT).features
        self.cnn_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.cnn_fc = nn.Linear(1280, cnn_embed_dim)

        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=cnn_embed_dim,
            nhead=num_heads,
            dim_feedforward=1024,
            dropout=0.1,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Classification head
        self.fc = nn.Sequential(
            nn.Linear(cnn_embed_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x: (batch, seq_len, 3, H, W)
        b, s, c, h, w = x.size()
        x = x.view(b * s, c, h, w)
        with torch.no_grad():
            f = self.cnn(x)
        f = self.cnn_pool(f).view(b * s, -1)
        f = self.cnn_fc(f)
        f = f.view(b, s, -1)
        t_out = self.transformer(f)
        out = self.fc(t_out[:, -1, :])
        return out


# ========================
# 6️⃣ Training setup
# ========================
model = CNN_Transformer().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10
best_acc = 0.0


# ========================
# 7️⃣ Training loop
# ========================
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for frames, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        frames, labels = frames.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(frames).squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_correct, val_total, val_loss = 0, 0, 0
    with torch.no_grad():
        for frames, labels in test_loader:
            frames, labels = frames.to(device), labels.to(device)
            outputs = model(frames).squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = (outputs > 0.5).float()
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = 100 * val_correct / val_total
    val_loss /= len(test_loader)

    print(f"\nEpoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_cnn_transformer.pth")
        print("✅ Saved best model")

print(f"\n🎯 Training complete! Best Validation Accuracy: {best_acc:.2f}%")


✅ Device: cuda
✅ Train samples: 61
✅ Test samples: 16


Epoch 1/10: 100%|██████████| 31/31 [00:27<00:00,  1.12it/s]



Epoch [1/10]
Train Loss: 0.7982 | Train Acc: 54.10%
Val Loss: 0.6592 | Val Acc: 56.25%
✅ Saved best model


Epoch 2/10: 100%|██████████| 31/31 [00:23<00:00,  1.34it/s]



Epoch [2/10]
Train Loss: 0.6627 | Train Acc: 54.10%
Val Loss: 0.6323 | Val Acc: 56.25%


...


Epoch [8/10]
Train Loss: 0.5823 | Train Acc: 67.21%
Val Loss: 0.3581 | Val Acc: 81.25%


Epoch 9/10: 100%|██████████| 31/31 [00:21<00:00,  1.44it/s]



Epoch [9/10]
Train Loss: 0.5561 | Train Acc: 72.13%
Val Loss: 0.3538 | Val Acc: 87.50%
✅ Saved best model


Epoch 10/10: 100%|██████████| 31/31 [00:17<00:00,  1.82it/s]



Epoch [10/10]
Train Loss: 0.4241 | Train Acc: 85.25%
Val Loss: 0.3609 | Val Acc: 81.25%

🎯 Training complete! Best Validation Accuracy: 87.50%


In [ ]:
model.eval()  # set model to evaluation mode
correct = 0
total = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for frames, labels in test_loader:
        frames, labels = frames.to(device), labels.to(device)
        outputs = model(frames).squeeze(1)       # shape: (batch_size,)
        preds = (outputs > 0.5).float()          # binary threshold
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 81.25%


In [ ]:
for i, (pred, label) in enumerate(zip(all_preds, all_labels)):
    print(f"Sample {i+1}: Predicted = {int(pred)}, Actual = {int(label)}")


Sample 1: Predicted = 0, Actual = 0
Sample 2: Predicted = 0, Actual = 0
Sample 3: Predicted = 0, Actual = 0
Sample 4: Predicted = 0, Actual = 0
Sample 5: Predicted = 1, Actual = 0
Sample 6: Predicted = 0, Actual = 0
Sample 7: Predicted = 0, Actual = 0
Sample 8: Predicted = 0, Actual = 0
Sample 9: Predicted = 0, Actual = 0
Sample 10: Predicted = 1, Actual = 1
Sample 11: Predicted = 1, Actual = 1
Sample 12: Predicted = 0, Actual = 1
Sample 13: Predicted = 0, Actual = 1
Sample 14: Predicted = 1, Actual = 1
Sample 15: Predicted = 1, Actual = 1
Sample 16: Predicted = 1, Actual = 1


In [ ]:
misclassified = [(i, pred, label) for i, (pred, label) in enumerate(zip(all_preds, all_labels)) if pred != label]

print(f"Misclassified samples: {len(misclassified)}")
for idx, pred, label in misclassified:
    print(f"Sample {idx}: Predicted = {int(pred)}, Actual = {int(label)}")


Misclassified samples: 3
Sample 4: Predicted = 1, Actual = 0
Sample 11: Predicted = 0, Actual = 1
Sample 12: Predicted = 0, Actual = 1


In [ ]:
from PIL import Image
import torch
from torchvision import transforms

def load_clip_frames(folder_path, seq_len=16, transform=None):
    """
    Load frames from a folder and apply transformations.
    Returns: Tensor of shape (1, seq_len, 3, H, W)
    """
    frame_files = sorted(os.listdir(folder_path))[:seq_len]  # take first seq_len frames
    frames = [Image.open(os.path.join(folder_path, f)).convert("RGB") for f in frame_files]

    if transform:
        frames = transform(frames)  # (seq_len, 3, H, W)
    frames = frames.unsqueeze(0)    # add batch dimension: (1, seq_len, 3, H, W)
    return frames


In [ ]:
model = CNN_Transformer().to(device)
model.load_state_dict(torch.load("best_cnn_transformer.pth", map_location=device))
model.eval()


CNN_Transformer(
  (cnn): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1

In [ ]:
# Example: path to a new clip folder
clip_path = "/content/drive/MyDrive/Frames_seqential_split/test/0/skeleton_slow_resized_baby_motion10_25fps_clip1"

# Load frames
frames = load_clip_frames(clip_path, seq_len=240, transform=VideoTransform(train=False))
frames = frames.to(device)

# Predict
with torch.no_grad():
    output = model(frames).squeeze(1)         # shape: (1,)
    pred_label = (output > 0.5).float().item()  # 0 or 1

print(f"Prediction: {'Normal' if pred_label==0 else 'Abnormal'}")


Prediction: Normal


In [ ]:
# Example: path to a new clip folder
clip_path = "/content/drive/MyDrive/Frames_seqential_split/test/1/skeleton_slow_resized_baby_motion8_25fps_clip1"

# Load frames
frames = load_clip_frames(clip_path, seq_len=240, transform=VideoTransform(train=False))
frames = frames.to(device)

# Predict
with torch.no_grad():
    output = model(frames).squeeze(1)         # shape: (1,)
    pred_label = (output > 0.5).float().item()  # 0 or 1

print(f"Prediction: {'Normal' if pred_label==0 else 'Abnormal'}")


Prediction: Abnormal


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import torch

# Load best saved model
model.load_state_dict(torch.load("best_cnn_transformer.pth"))
model.eval()

all_labels = []
all_preds = []

with torch.no_grad():
    for frames, labels in test_loader:
        frames, labels = frames.to(device), labels.to(device)
        outputs = model(frames).squeeze()
        preds = (outputs > 0.5).float()
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Convert to arrays
import numpy as np
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)


In [ ]:
print("✅ Model Evaluation Metrics")
print("Accuracy :", accuracy_score(all_labels, all_preds))
print("Precision:", precision_score(all_labels, all_preds))
print("Recall   :", recall_score(all_labels, all_preds))
print("F1-Score :", f1_score(all_labels, all_preds))

# Optional detailed report
print("\nClassification Report:\n", classification_report(all_labels, all_preds))

# Confusion matrix
print("\nConfusion Matrix:\n", confusion_matrix(all_labels, all_preds))


✅ Model Evaluation Metrics
Accuracy : 0.875
Precision: 0.8571428571428571
Recall   : 0.8571428571428571
F1-Score : 0.8571428571428571

Classification Report:
               precision    recall  f1-score   support

         0.0       0.89      0.89      0.89         9
         1.0       0.86      0.86      0.86         7

    accuracy                           0.88        16
   macro avg       0.87      0.87      0.87        16
weighted avg       0.88      0.88      0.88        16


Confusion Matrix:
 [[8 1]
 [1 6]]
